In [1]:
import time
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import imutils
import matplotlib.image as mpimg
from collections import OrderedDict
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages

from skimage import io, transform
import math
from math import * 
import xml.etree.ElementTree as ET
from skimage.transform import AffineTransform, warp
from skimage.transform import rotate as rotate_transform
from skimage.util import random_noise
from skimage.filters import gaussian
from sklearn.linear_model import LinearRegression
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pickle

In [2]:
class Transforms():
    def __init__(self):
        pass
    
    def rotate(self, image, params):

        angle = params['rotation_range'][0]
        angle = (random.uniform(0,1))*random.choice([-1,1])*angle
        transformation_matrix = torch.tensor([
            [+cos(radians(angle)), -sin(radians(angle))], 
            [+sin(radians(angle)), +cos(radians(angle))]
        ])

        image = rotate_transform(np.array(image), angle = angle, mode = 'edge')

        # PIL expects RGB images to be uint with ranges from 0 to 255 so we have to convert it to a type that PIL can excpect ie a uint from 0 to 255 
        return Image.fromarray((image * 255).astype(np.uint8))

    def translation(self, image,  params):
        image_shape = np.array(image).shape
        ty = random.uniform(params['height_shift_range'][0]*image_shape[0],          
                            params['height_shift_range'][1]*image_shape[0])
        tx = random.uniform(params['width_shift_range'][0]*image_shape[1],
                            params['width_shift_range'][1]*image_shape[1] )

        
        horizontal_shift =  tx*random.choice([-1,1])
        vertical_shift = ty*random.choice([-1,1])
        horizontal_shift_normalised = horizontal_shift/image_shape[1]
        vertical_shift_normalised =  vertical_shift/image_shape[0]

        transform = AffineTransform(translation=(-horizontal_shift,-vertical_shift))

        image = warp(np.array(image),transform,mode='edge')


  
        # PIL expects RGB images to be uint with ranges from 0 to 255 so we have to convert it to a type that PIL can excpect ie a uint from 0 to 255 
        return Image.fromarray((image * 255).astype(np.uint8))
        
    def resize(self, image, img_size):
        image = TF.resize(image, img_size)
        return image

    def zoom(self, image, params):

        img_shape = np.array(image).shape
        zoom = random.uniform(params['zoom_range'][0],params['zoom_range'][1])
        image = TF.resize(image,(int(img_shape[0]*zoom), int(img_shape[1]*zoom)) )
        scale_transform = torch.tensor([[zoom, 0], 
                                        [0, zoom]])

        
        return image

    def color_jitter(self, image):
        color_jitter = transforms.ColorJitter(brightness=0.3, 
                                              contrast=0.3,
                                              saturation=0.3, 
                                              hue=0.1)
        image = color_jitter(image)
        return image
    
    def __call__(self, image, params, image_size):

        # set checked image and landmark to landmark_ and image_ (this is for making sure we use the last checked tranformed instead of wrongly tranformed to do the following               # tranform)
        
        # -----------------------
        image_ = Image.fromarray(image.copy())

        # -----------------------

        # ZOOM
        image  = self.zoom(image_,  params)
        

        # RESIZE

        image = self.resize(image, (image_size, image_size))

        # ----------------------
        #image_, landmarks_ = self.color_jitter(image_, landmarks_)
        # ----------------------
        
        # ROTATE
        image = self.rotate(image,  params)


        # ----------------------

        image = image
        # ----------------------

        # TRANSLATION
        image= self.translation(image, params)

 
        
        image = TF.to_tensor(image)
        # the following tranform normalises each channel to have a mean at 0.5 and std of 0.5 / NOTE: NOT sure if this is theoreticlly better, should check this
        image = TF.normalize(image, [0.5], [0.5])
        return image

In [3]:
class LandmarksDataset():

    def __init__(self, transform=None,zoom = [1.0 - 0.03258157476873315, 1.0 + 0.03258157476873315], rotation = [22], height_shift= [0,0.03003200603616672], width_shift= [0,0.03003200603616672 ]):

        # targets 0
        filenames1 = os.listdir('C:/Users/19480105/OneDrive - Stellenbosch University/Documents/1. Research Project/Data/tier1_training_set/Missing_landmarkwings_L/')
        # targets 1
        filenames3 = os.listdir('C:/Users/19480105/OneDrive - Stellenbosch University/Documents/1. Research Project/Data/tier1_training_set/goodwingsv20-21/')
    
        self. tranform = transform
        self.zoom = zoom
        self.rotation = rotation
        self.height_shift = height_shift
        self.width_shift = width_shift
        self.image_filenames = []
        self.targets = []
        self.image_size = 224
        self.transform = transform
        self.image_dir = 'C:/Users/19480105/OneDrive - Stellenbosch University/Documents/1. Research Project/Data/tier1_training_set/Missing_landmarkwings_L/'
        
        self.image_dir3 = 'C:/Users/19480105/OneDrive - Stellenbosch University/Documents/1. Research Project/Data/tier1_training_set/goodwingsv20-21/'
        self.TransF_ = True

       # ------------------- Append left wings data to dataset class ------------

        for filename in filenames1:
            self.image_filenames.append(os.path.join(self.image_dir, filename))
            self.targets.append(1)

            

        # ------------------ Append flipped right wings data to dataset class-----


        #for filename in filenames2[:]:
        #    self.mirror(image)
        #    self.targets.append(1)
        #    self.image_filenames.append(os.path.join(self.image_dir2, filename))

        #num = len(self.targets.copy())
        for filename in filenames3:
            self.targets.append(0)
            self.image_filenames.append(os.path.join(self.image_dir3, filename))



        # ----------------------

    def TransF(self):
        self.TransF_ = True
    def NoTransF(self):
        self.TransF_ = False
    def resize(self,size):
        self.image_size = size
    def set_params(self, zoom = [0.95, 0.105], rotation = [10], height_shift= [0,0.05], width_shift= [0,0.05]):
        self.zoom = zoom
        self.rotation = rotation
        self.height_shift = height_shift
        self.width_shift = width_shift
    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        params = {'zoom_range': self.zoom, 'rotation_range':self.rotation, 'height_shift_range': self.height_shift, 'width_shift_range': self.width_shift }
        image_ = plt.imread(self.image_filenames[index])
        target = torch.tensor(self.targets[index])

        image = plt.imread(self.image_filenames[index])

        
        if self.transform and self.TransF_:
            
            image = self.transform(image_, params, self.image_size)

        else:
            img_shape = image.copy().shape
            image = Image.fromarray(image)
            image = TF.resize(image, (self.image_size,self.image_size))
       
            image = TF.to_tensor(image)
            # the following tranform normalises each channel to have a mean at 0.5 and std of 0.5 / NOTE: NOT sure if this is theoreticlly better, should check this
            image = TF.normalize(image, [0.5], [0.5])

        return image, target

DataSet = LandmarksDataset(Transforms())

In [4]:
class vgg16_bn_(nn.Module):
    def __init__(self,num_classes=1):
        super().__init__()
        self.model_name='vgg16_bn'
        self.model=models.vgg16_bn(pretrained=True)
        #self.model.conv1=nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.classifier=nn.Linear(self.model.classifier[0].in_features, num_classes)
        
    def forward(self, x):
        x = torch.sigmoid(self.model(x))
        return x

In [5]:
vgg16_bn_()

C:\Users\nuhrr\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\nuhrr\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


vgg16_bn_(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=0, di

In [6]:
def accuracy(predictions, y):
    return(sum((predictions.round() == y)) / float(len(y))).item()


    # helper functions
import sys

def print_overwrite(step, total_step, loss, operation):
    sys.stdout.write('\r')
    if operation == 'train':
        sys.stdout.write("Train Steps: %d/%d  Loss: %.6f " % (step, total_step, loss))   
    else:
        sys.stdout.write("Valid Steps: %d/%d  Loss: %.6f " % (step, total_step, loss))
        
    sys.stdout.flush()

In [7]:
DataSet.TransF()
DataSet.resize(224)
dataset = DataSet
# split the dataset into validation and test sets
len_valid_test_set = int(0.2*len(dataset)) # 60% training, 20% validation, 20% testing

len_train_set = len(dataset) - len_valid_test_set*2

print("The length of Train set is {}".format(len_train_set))
print("The length of Valid set is {}".format(len_valid_test_set))
print("The length of Valid set is {}".format(len_valid_test_set))

train_dataset , valid_dataset, test_dataset  = torch.utils.data.random_split(dataset , [len_train_set, len_valid_test_set, len_valid_test_set], generator=torch.Generator().manual_seed(42))

# shuffle and batch the datasets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=15, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=15, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=15, shuffle=True)

The length of Train set is 720
The length of Valid set is 240
The length of Valid set is 240


In [10]:
# feature transfer learning (none of th weights are frozen)

results = {'acc_train': [], 'acc_val': [], 'loss_train': [], 'loss_val':[], 'time': []}
network = vgg16_bn_()

# TRAIN
torch.autograd.set_detect_anomaly(True)

criterion = nn.BCELoss()
optimizer = optim.Adam(network.parameters(), lr = 0.0001)
loss_min = np.inf
num_epochs = 30

start_time = time.time()
for epoch in range(1,num_epochs+1):
    acc_train = 0
    acc_valid = 0
    running_acc = 0
    loss_train = 0
    loss_valid = 0
    running_loss = 0
    
    network.train()
    for step in range(1,len(train_loader)+1):

        images, targets = next(iter(train_loader))
        targets = targets.float()
    
        predictions = network(images).flatten()

        optimizer.zero_grad()
    
        
        # find the loss for the current step
    
        loss_train_step = criterion(predictions, targets)


        acc_train_step = accuracy(predictions, targets)


        
        # calculate the gradients

        loss_train_step.backward()
    
        
        # update the parameters

        optimizer.step()
        acc_train += acc_train_step
        loss_train += loss_train_step.item()
        
        running_acc  = acc_train/step
        running_loss = loss_train/step

        
        print_overwrite(step, len(train_loader), running_loss, 'train')
        
    network.eval() 
    with torch.no_grad():
        
        for step in range(1,len(valid_loader)+1):
            
            
            images, targets = next(iter(valid_loader))
            targets = targets.float()
        
            predictions = network(images).flatten()

            # find the loss for the current step
            loss_valid_step = criterion(predictions, targets)

            acc_valid_step = accuracy(predictions, targets)

            acc_valid += acc_valid_step
            running_acc = acc_valid/step
            loss_valid += loss_valid_step.item()
            running_loss = loss_valid/step

            print_overwrite(step, len(valid_loader), running_loss, 'valid')
    
    loss_train /= len(train_loader)
    loss_valid /= len(valid_loader)
    acc_train /= len(train_loader)
    acc_valid /= len(valid_loader)
    
    
    print('\n--------------------------------------------------')
    print('Epoch: {}  Train Loss: {:.4f}  Valid Loss: {:.4f}'.format(epoch, loss_train, loss_valid))
    print('--------------------------------------------------')
    print('Epoch: {}  Train acc: {:.5f}  Valid acc: {:.5f}'.format(epoch, acc_train, acc_valid))
    print('--------------------------------------------------')
    results['loss_train'].append(loss_train)
    results['loss_val'].append(loss_valid)
    results['acc_train'].append(acc_train)
    results['acc_val'].append(acc_valid)

    if loss_valid < loss_min:
        loss_min = loss_valid
        torch.save(network.state_dict(), 'C:/Users/nuhrr/OneDrive - Stellenbosch University/Documents/1. Research Project/Models/model_vgg16_bn_classifer_finetune.pth') 
        print("\nMinimum Validation Loss of {:.4f} at epoch {}/{}".format(loss_min, epoch, num_epochs))
        print('Model Saved\n')
print('Training Complete')
print("Total Elapsed Time : {} s".format(time.time()-start_time))
results['time'].append(time.time()-start_time)
del(network)
del(images)
del(targets)
del(predictions)
#torch.empty_cache() #torch.cuda.empty_cache()



Valid Steps: 16/16  Loss: 0.045525 
--------------------------------------------------
Epoch: 1  Train Loss: 0.2292  Valid Loss: 0.0455
--------------------------------------------------
Epoch: 1  Train acc: 0.91250  Valid acc: 0.98750
--------------------------------------------------

Minimum Validation Loss of 0.0455 at epoch 1/30
Model Saved

Valid Steps: 16/16  Loss: 0.008297 
--------------------------------------------------
Epoch: 2  Train Loss: 0.0326  Valid Loss: 0.0083
--------------------------------------------------
Epoch: 2  Train acc: 0.99167  Valid acc: 1.00000
--------------------------------------------------

Minimum Validation Loss of 0.0083 at epoch 2/30
Model Saved

Valid Steps: 16/16  Loss: 0.004669 
--------------------------------------------------
Epoch: 3  Train Loss: 0.0149  Valid Loss: 0.0047
--------------------------------------------------
Epoch: 3  Train acc: 0.99583  Valid acc: 1.00000
--------------------------------------------------

Minimum Valida

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/nuhrr/OneDrive - Stellenbosch University/Documents/1. Research Project/Models/training_losses/model_vgg16_bn_classifer_finetune_trainingdata.pkl'

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/nuhrr/OneDrive - Stellenbosch University/Documents/1. Research Project/Models/training_losses/model_vgg16_bn_classifer_finetune_trainingdata.pkl'

In [11]:
f = open("C:/Users/19480105/OneDrive - Stellenbosch University/Documents/1. Research Project/Models/training_losses/model_vgg16_bn_classifer_finetune_trainingdata.pkl","wb")
pickle.dump(results,f)
f.close()

In [ ]:
# feature transfer learning (weights are frozen)

results = {'acc_train': [], 'acc_val': [], 'loss_train': [], 'loss_val':[], 'time': []}
network = vgg16_bn_()
for param in network.model.features.parameters():
    param.requires_grad = False
# TRAIN
torch.autograd.set_detect_anomaly(True)

criterion = nn.BCELoss()
optimizer = optim.Adam(network.parameters(), )
loss_min = np.inf
num_epochs = 50

start_time = time.time()
for epoch in range(1,num_epochs+1):
    acc_train = 0
    acc_valid = 0
    running_acc = 0
    loss_train = 0
    loss_valid = 0
    running_loss = 0
    
    network.train()
    for step in range(1,len(train_loader)+1):

        images, targets = next(iter(train_loader))
        images = images.float()
        targets = targets.float()
    
        predictions = network(images).flatten()

        optimizer.zero_grad()
    
        
        # find the loss for the current step
    
        loss_train_step = criterion(predictions, targets)


        acc_train_step = accuracy(predictions, targets)


        
        # calculate the gradients

        loss_train_step.backward()
    
        
        # update the parameters

        optimizer.step()
        acc_train += acc_train_step
        loss_train += loss_train_step.item()
        
        running_acc  = acc_train/step
        running_loss = loss_train/step

        
        print_overwrite(step, len(train_loader), running_loss, 'train')
        
    network.eval() 
    with torch.no_grad():
        
        for step in range(1,len(valid_loader)+1):
            
            
            images, targets = next(iter(valid_loader))
            images = images.float()
            targets = targets.float()
        
            predictions = network(images).flatten()

            # find the loss for the current step
            loss_valid_step = criterion(predictions, targets)

            acc_valid_step = accuracy(predictions, targets)

            acc_valid += acc_valid_step
            running_acc = acc_valid/step
            loss_valid += loss_valid_step.item()
            running_loss = loss_valid/step

            print_overwrite(step, len(valid_loader), running_loss, 'valid')
    
    loss_train /= len(train_loader)
    loss_valid /= len(valid_loader)
    acc_train /= len(train_loader)
    acc_valid /= len(train_loader)
    
    
    print('\n--------------------------------------------------')
    print('Epoch: {}  Train Loss: {:.4f}  Valid Loss: {:.4f}'.format(epoch, loss_train, loss_valid))
    print('--------------------------------------------------')
    results['loss_train'].append(loss_train)
    results['loss_train'].append(loss_valid)
    results['acc_train'].append(acc_train)
    results['acc_val'].append(acc_valid)

    if loss_valid < loss_min:
        loss_min = loss_valid
        torch.save(network.state_dict(), 'C:/Users/nuhrr/OneDrive - Stellenbosch University/Documents/1. Research Project/Models/model_vgg16_bn_classifer_fixedfeatures.pth') 
        print("\nMinimum Validation Loss of {:.4f} at epoch {}/{}".format(loss_min, epoch, num_epochs))
        print('Model Saved\n')
print('Training Complete')
print("Total Elapsed Time : {} s".format(time.time()-start_time))
results['time'].append(time.time()-start_time)
del(network)
del(images)
del(targets)
del(predictions)
#torch.cuda.empty_cache()

    
f = open("C:/Users/nuhrr/OneDrive - Stellenbosch University/Documents/1. Research Project/Models/training_losses/model_vgg16_bn_classifer_fixedfeatures_trainingdata.pkl","wb")
pickle.dump(results,f)
f.close()